# Movie Recommendation System

## Collaborative Filtering
#### Item-to-Item Collaborative Filtering
- Select a user with the movies the user has watched
- Based on his rating to movies, find the top X neighbours
- Get the watched movie record of the user for each neighbour.
- Calculate a similarity score using some formula
- Recommend the items with the highest score
- Rated score below 3 (Dislike) and 3,4 or 5 (Like)

In [1]:
import pandas as pd
from scipy import sparse

### Load Data

In [2]:
rating_df = pd.read_csv('movie_rating.csv')
movie_df = pd.read_csv('movies.csv')

### List movie rated

In [3]:
rating_df.head()

,id,user_id,movie_id,rating
0,270,14,21,5
1,271,15,21,3
2,272,15,34,3
3,273,15,23,5
4,276,14,23,2


###  All movies

In [4]:
movie_df = movie_df.drop(['genre','movie_logo'],axis=1) # remove colomn genre, movie_logo
movie_df.head()

,id,title
0,18,12 Angry Men
1,19,12 Years a Slave
2,20,A Beautiful Mind
3,21,Avatar
4,22,Avengers: Infinity War


### Merge movie_df with rating_df

In [5]:
movie_rating_df = rating_df.merge(movie_df,left_on='movie_id', right_on='id')
print('Shape:', movie_rating_df.shape)
movie_rating_df.head()

Shape: (6, 6)


,id_x,user_id,movie_id,rating,id_y,title
0,270,14,21,5,21,Avatar
1,271,15,21,3,21,Avatar
2,272,15,34,3,34,Frozen
3,273,15,23,5,23,Avengers: Endgame
4,276,14,23,2,23,Avengers: Endgame


### List all user rated on movies as Pivot Table

In [6]:
user_rating_df= movie_rating_df.pivot_table(index=['user_id'],columns=['movie_id'],values='rating')
user_rating_df.head()

movie_id,21,23,34,54
user_id,,,,
14,5.0,2.0,NaN,NaN
15,3.0,5.0,3.0,NaN
16,NaN,NaN,NaN,5.0


Fill remaining NaN with 0

In [7]:
user_rating_df = user_rating_df.fillna(0,axis=1)
user_rating_df.head()

movie_id,21,23,34,54
user_id,,,,
14,5.0,2.0,0.0,0.0
15,3.0,5.0,3.0,0.0
16,0.0,0.0,0.0,5.0


Let's remove movie which have less than 10 who rated it, and fill remaining NaN with 0
if we have lots of movie rated, so we show movie's rated over 10 only

In [8]:
# print("Before: ",user_rating_df.shape)

# Let's remove movie which have less than 10 who rated it, and fill remaining NaN with 0
# user_rating_df = user_rating_df.dropna(thresh=10, axis=1).fillna(0,axis=1)
# userRatings.fillna(0, inplace=True)
# print("After: ",user_rating_df.shape)

In [9]:
### Let's build our Pearson Correlation Similarity Matrix

In [10]:
corrMatrix = user_rating_df.corr(method='pearson')
corrMatrix.head()

movie_id,21,23,34,54
movie_id,,,,
21,1.000000,0.500000,0.114708,-0.917663
23,0.500000,1.000000,0.917663,-0.802955
34,0.114708,0.917663,1.000000,-0.500000
54,-0.917663,-0.802955,-0.500000,1.000000


### Let's Make Recommendations
- score less than 3 is Dislike, so move that movie to down(that why use -2.5)
- score 3, 4 or 5 is Like, so show that movie to top

In [11]:
def get_similar_movie(movie_id, rating_score):
    similar_ratings = corrMatrix[movie_id] * (rating_score-2.5) 
    similar_ratings = similar_ratings.sort_values(ascending=False) # Order score DESC
    #print(type(similar_ratings))
    return similar_ratings

### Let's test rating movie here

In [12]:
# rating_movie = [(movie_id, rating_scor)]
my_rating_movie = [
    (57, 5),
    (58, 3),
    (59,1)
]
similar_movies = pd.DataFrame()
for movie,rating_score in my_rating_movie:
    similar_movies = similar_movies.append(get_similar_movie(movie, rating_score),ignore_index = True)

similar_movies.head(10) # show limit 10 rows

KeyError: 57